时间序列的应用场景：

    时间戳（timestamp），特定的时刻。 
    固定时期（period），如2007年1月或2010年全年。 
    时间间隔（interval），由起始和结束时间戳表示。时期（period）可以被看做间隔（interval）的特例。 
    实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。例如，从放入烤箱时起，每秒钟饼干的直径。 

本章主要讲解前3种时间序列。许多技术都可用于处理实验型时间序列，其索引可能是一个整数或浮点数（表示从实验开始算起已经过去的时间）。最简单也最常见的时间序列都是用时间戳进行索引的。

pandas提供了许多内置的时间序列处理工具和数据算法。因此，你可以高效处理非常大的时间序列，轻松地进行切片/切块、聚合、对定期/不定期的时间序列进行重采样等。有些工具特别适合金融和经济应用，你当然也可以用它们来分析服务器日志数据。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime #导入惯例
from datetime import timedelta

pd.set_option('display.max_rows', 10)

# 11.1 日期和时间数据类型及工具

Python标准库包含用于日期（date）和时间（time）数据的数据类型，而且还有日历方面的功能。我们主要会用到datetime、time以及calendar模块。datetime.datetime（也可以简写为datetime）是用得最多的数据类型。

常见时间数据类型：
    
    date    日期
    time    时、分、秒和毫秒
    datetime    储存日期和时间
    timedelta    两个datetime之间的时间差

datetime以毫秒形式存储日期和时间。

timedelta表示两个datetime对象之间的时间差。可以给datetime对象加上（或减去）一个或多个timedelta，这样会产生一个新对象。

In [6]:
from datetime import datetime
from datetime import timedelta

now = datetime.now()
now
#现在的时间，从年精确到毫秒。

datetime.datetime(2019, 4, 25, 15, 0, 49, 890754)

In [4]:
type(now) #datetime对象

datetime.datetime

In [7]:
now+timedelta(6)
#timedalta()以天为单位传入时差。

datetime.datetime(2019, 5, 1, 15, 0, 49, 890754)

### 11.1.1 字符串str和时间datetime的相互转换

利用str或strftime方法（传入一个格式化字符串），datetime对象和pandas的Timestamp对象可以被格式化为字符串。

datetime.strptime可以将字符串转换为日期。

关于datetime的格式化定义见Page
        
        %Y    4位数的年份
        %y    2位数的年份 
        %m    2位数的月份[1,12]
        %d    2位数的日[1,31]
        %w    用整数表示的星期几[0,6]，0是星期天。
        %W    每年的第几周[00,53]，星期一被认为是第一天。若星期天被认为是第一天，使用%U。
        
        %F    %Y-%d-%m 的简写形式，如2014-08-30
        %D    %m/%d/%y 的简写形式，如08/30/14

In [9]:
stamp = datetime(2011, 1, 3)
str(stamp)
#对datetime对象使用数据类型函数str，转换成字符串。这里精确到时、分、秒。

'2011-01-03 00:00:00'

In [10]:
stamp.strftime('%Y-%m-%d')
#对datetime对象使用strftime方法将其转换成字符串

'2011-01-03'

In [11]:
value = '2011-01-03'
datetime.strptime(value, '%Y-%m-%d')
#使用顶级方法strptime可以将字符串转换为datetime。注意字符串格式对应(-,/)

datetime.datetime(2011, 1, 3, 0, 0)

In [12]:
datestrs = ['7/6/2011', '8/6/2011']
[datetime.strptime(x, '%m/%d/%Y') for x in datestrs]

#列表推导式。生成一个列表，元素为两个datetime对象。

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

上述方法需要对字符串格式有明确要求。对于一些常见的日期格式可以用dateutil这个第三方包中的parser.parse方法：

parse有一个dayfirst=True，可以是的日期在月份前面。

In [2]:
from dateutil.parser import parse #引入惯例

#parser.parse可以对常见格式的日期格式，进行字符串到datetime的转换。

In [14]:
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

In [15]:
parse('2011/01/03')

datetime.datetime(2011, 1, 3, 0, 0)

In [16]:
parse('01/03/11')

datetime.datetime(2011, 1, 3, 0, 0)

In [17]:
parse('3pm , Jan 3rd, 2011')

datetime.datetime(2011, 1, 3, 15, 0)

pandas通常是用于处理成组日期的，不管这些日期是DataFrame的轴索引还是列。

pd.to_datetime方法可以解析多种不同的日期表示形式。对标准日期格式（如ISO8601）的解析非常快：

In [18]:
datestrs = ['2011-07-06 12:00:00', '2011-08-06 00:00:00']
pd.to_datetime(datestrs)  

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

# 11.2 时间序列基础

pandas最基本的时间序列类型就是以时间戳（通常以Python字符串或datatime对象表示）为索引的Series。

In [3]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]
#dates是一个以datetime对象为元素的列表

ts = pd.Series(np.random.randn(6), index=dates)
#通过pd.Series方法将列表转换为Series对象。注意索引使用了dates这个以datetime对象为元素的列表

ts

2011-01-02   -1.110312
2011-01-05    0.581693
2011-01-07    1.676844
2011-01-08    0.745587
2011-01-10    0.384457
2011-01-12    0.917605
dtype: float64

In [4]:
ts.index
#这个时间序列的索引，是一个DatetimeIndex列表

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [5]:
ts.index[0]
#可以看到DatetimeIndex列表的元素是TimeStamp对象，可以转换成datetime对象。

Timestamp('2011-01-02 00:00:00')

总结：
    
    以datetime对象为元素的列表list——pandas对象的索引，使其成为DatetimeIndex列表——每个元素为TimeStamp对象（不是datetime对象）

## 11.2.1 索引、切片和子集选取

以Series为例讲解，适用于DataFrame对象。

可以使用通用的标签索引方法索引Series和DataFrame对象，也可以使用特殊的方法即传入一个日期格式字符串：

    使用默认的整数编号
    使用标签内容（即TimeStamp对象）
    使用日期格式字符串

In [6]:
ts

2011-01-02   -1.110312
2011-01-05    0.581693
2011-01-07    1.676844
2011-01-08    0.745587
2011-01-10    0.384457
2011-01-12    0.917605
dtype: float64

In [8]:
ts[3]
#标签的系统默认整数编号

0.745586747295785

In [9]:
stamp = ts.index[3]
ts[stamp]
#根据标签的具体值（这里是一个TimeStamp对象）来索引

0.745586747295785

In [11]:
ts['20110108']
#传入一个可以被解释为日期的字符串

0.745586747295785

选取子集的方法，时间序列也可以使用一些通用方法。但由于我们往往想根据时间特性进行索引：

    日期格式字符串(有条件的分隔符)
    Datetime对象
    TimeStamp对象

In [12]:
longer_ts = pd.Series(np.random.randn(1000),
                      index=pd.date_range('1/1/2000', periods=1000))
longer_ts
#这里生成一个长时间序列

2000-01-01   -0.576973
2000-01-02   -0.526006
2000-01-03   -1.175904
2000-01-04   -0.420021
2000-01-05    0.158667
                ...   
2002-09-22   -0.004952
2002-09-23    0.633606
2002-09-24    0.027607
2002-09-25   -0.272760
2002-09-26    0.179185
Freq: D, Length: 1000, dtype: float64

In [13]:
longer_ts['2001']
#传入”年“来选取整年内的数据子集。这个方法的实质其实是”传入日期格式的字符串“

2001-01-01    0.368309
2001-01-02   -0.370220
2001-01-03    0.510407
2001-01-04    1.078665
2001-01-05    0.538448
                ...   
2001-12-27   -2.190308
2001-12-28    0.468285
2001-12-29   -0.648500
2001-12-30    0.745875
2001-12-31    0.631644
Freq: D, Length: 365, dtype: float64

In [20]:
longer_ts['2001-05']
#注意：对传入的日期格式字符串有格式要求，即必须有分隔符（可以是-，/， (空格)）。

2001-05-01    1.266337
2001-05-02   -0.400168
2001-05-03   -1.609220
2001-05-04    0.052956
2001-05-05    0.991742
                ...   
2001-05-27    0.618851
2001-05-28    1.309880
2001-05-29    1.485027
2001-05-30    0.001195
2001-05-31   -0.570798
Freq: D, Length: 31, dtype: float64

In [21]:
longer_ts['20010501':'20010701']
#切片。可以通过字符串日期、datetime和TimeStamp对象进行切片。

2001-05-01    1.266337
2001-05-02   -0.400168
2001-05-03   -1.609220
2001-05-04    0.052956
2001-05-05    0.991742
                ...   
2001-06-27    1.834563
2001-06-28   -1.081911
2001-06-29   -0.463131
2001-06-30    0.161829
2001-07-01   -0.369285
Freq: D, Length: 62, dtype: float64

In [22]:
dates = pd.date_range('1/1/2000', periods=100, freq='W-WED')
#生成时间索引。W-WED表示每周的周三

long_df = pd.DataFrame(np.random.randn(100, 4),
                       index=dates,
                       columns=['Colorado', 'Texas',
                                'New York', 'Ohio'])

long_df

,Colorado,Texas,New York,Ohio
2000-01-05,1.022443,-0.162260,-0.646437,-0.263800
2000-01-12,0.423886,-1.683049,-0.853287,0.076513
2000-01-19,-0.414670,0.196938,-1.330608,0.523249
2000-01-26,0.215733,0.467299,-0.233258,-0.519130
2000-02-02,0.788068,1.064270,-0.542895,-0.840126
...,...,...,...,...
2001-10-31,0.808964,-0.146502,2.228187,1.534247
2001-11-07,0.207702,1.030757,0.086883,-0.036157
2001-11-14,1.780738,0.984026,-0.672067,-0.984152
2001-11-21,0.239739,-0.667554,0.430946,0.145673


In [23]:
long_df.index

DatetimeIndex(['2000-01-05', '2000-01-12', '2000-01-19', '2000-01-26',
               '2000-02-02', '2000-02-09', '2000-02-16', '2000-02-23',
               '2000-03-01', '2000-03-08', '2000-03-15', '2000-03-22',
               '2000-03-29', '2000-04-05', '2000-04-12', '2000-04-19',
               '2000-04-26', '2000-05-03', '2000-05-10', '2000-05-17',
               '2000-05-24', '2000-05-31', '2000-06-07', '2000-06-14',
               '2000-06-21', '2000-06-28', '2000-07-05', '2000-07-12',
               '2000-07-19', '2000-07-26', '2000-08-02', '2000-08-09',
               '2000-08-16', '2000-08-23', '2000-08-30', '2000-09-06',
               '2000-09-13', '2000-09-20', '2000-09-27', '2000-10-04',
               '2000-10-11', '2000-10-18', '2000-10-25', '2000-11-01',
               '2000-11-08', '2000-11-15', '2000-11-22', '2000-11-29',
               '2000-12-06', '2000-12-13', '2000-12-20', '2000-12-27',
               '2001-01-03', '2001-01-10', '2001-01-17', '2001-01-24',
      

In [24]:
long_df['2001-05']

,Colorado,Texas,New York,Ohio
2001-05-02,-0.390783,-1.509089,-0.745683,1.205508
2001-05-09,-1.776554,0.332178,0.261635,-0.893066
2001-05-16,1.436377,-1.767451,-0.290507,-0.344847
2001-05-23,-0.267707,0.288780,-0.386526,-1.959422
2001-05-30,-1.215796,-0.123680,-0.160766,-1.620486


## 11.2.2 带有重复索引的时间序列

在某些应用场景中，可能会存在多个观测数据落在同一个时间点上的情况。

In [25]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000',
                          '1/2/2000', '1/3/2000'])

dup_ts = pd.Series(np.arange(5), index=dates)

dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

In [26]:
dup_ts.index.is_unique
#使用is_unique的方法检查是否唯一

False

如果想对具有非唯一时间戳的数据进行聚合，一个办法是使用groupby，并传入level=0：

In [27]:
grouped = dup_ts.groupby(level=0)
grouped

In [28]:
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

In [29]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 11.3 日期的范围、频率以及移动

pandas中的原生时间序列一般被认为是不规则的，也就是说，它们没有固定的频率。对于大部分应用程序而言，这是无所谓的。但是，它常常需要以某种相对固定的频率进行分析，比如每日、每月、每15分钟等（这样自然会在时间序列中引入缺失值）。幸运的是，pandas有一整套标准时间序列频率以及用于重采样、频率推断、生成固定频率日期范围的工具。

### 11.3.1 生成日期范围

pandas.date_range可用于根据指定的频率生成指定长度的DatetimeIndex：

    pandas.date_range有四个基本参数：pandas.date_range(*start*,*end*,*periods*,*freq*)
    
    常见的freq参数有：
        D    day    天
        B    Businessday    工作日
        H    Hour    小时
        T or min    Minute    分钟
        S    Second    秒
        M    MonthEnd    每月的最后一天
        BM  BusinessMonthEnd    每月的最后一个工作日
        MS  MonthBegin    每月的第一天
        BMS  BusinessMonthBegin    每月的第一个工作日
        W-MON, W-TUE  Week    每周的周几
        WOM-1MON  WeekOfMonth    每月的第几个周几。例如WOM-3FRI指每月第3个周五
        
    更多的freq参数，见P329.

In [32]:
index = pd.date_range('2012-03-01', '2012-06-01',freq='BM')
index

DatetimeIndex(['2012-03-30', '2012-04-30', '2012-05-31'], dtype='datetime64[ns]', freq='BM')

In [34]:
index = pd.date_range('2012-03-01', '2012-06-01',freq='MS')
index

DatetimeIndex(['2012-03-01', '2012-04-01', '2012-05-01', '2012-06-01'], dtype='datetime64[ns]', freq='MS')

### 11.3.2 频率和日期偏移量

pandas中的频率是由一个基础频率（base frequency）和一个乘数组成的。基础频率通常以一个字符串别名表示，比如"M"表示每月，"H"表示每小时。

常用的方法有：利用基础频率及其倍乘、频率字符串。

In [35]:
pd.date_range('2000-01-01', '2000-01-03 23:59', freq='4h')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4H')

同理，你也可以传入频率字符串（如"2h30min"），这种字符串可以被高效地解析为等效的表达式：

In [36]:
pd.date_range('2000-01-01', periods=10, freq='1h30min')

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

WOM（Week Of Month）是一种非常实用的频率类，它以WOM开头。它使你能获得诸如“每月第3个星期五”之类的日期：

In [42]:
rng = pd.date_range('2014-01-01', '2019-12-31', freq='WOM-4SUN')
rng

DatetimeIndex(['2014-01-26', '2014-02-23', '2014-03-23', '2014-04-27',
               '2014-05-25', '2014-06-22', '2014-07-27', '2014-08-24',
               '2014-09-28', '2014-10-26', '2014-11-23', '2014-12-28',
               '2015-01-25', '2015-02-22', '2015-03-22', '2015-04-26',
               '2015-05-24', '2015-06-28', '2015-07-26', '2015-08-23',
               '2015-09-27', '2015-10-25', '2015-11-22', '2015-12-27',
               '2016-01-24', '2016-02-28', '2016-03-27', '2016-04-24',
               '2016-05-22', '2016-06-26', '2016-07-24', '2016-08-28',
               '2016-09-25', '2016-10-23', '2016-11-27', '2016-12-25',
               '2017-01-22', '2017-02-26', '2017-03-26', '2017-04-23',
               '2017-05-28', '2017-06-25', '2017-07-23', '2017-08-27',
               '2017-09-24', '2017-10-22', '2017-11-26', '2017-12-24',
               '2018-01-28', '2018-02-25', '2018-03-25', '2018-04-22',
               '2018-05-27', '2018-06-24', '2018-07-22', '2018-08-26',
      

In [40]:
list(rng)

[Timestamp('2012-01-22 00:00:00', freq='WOM-4SUN'),
 Timestamp('2012-02-26 00:00:00', freq='WOM-4SUN'),
 Timestamp('2012-03-25 00:00:00', freq='WOM-4SUN'),
 Timestamp('2012-04-22 00:00:00', freq='WOM-4SUN'),
 Timestamp('2012-05-27 00:00:00', freq='WOM-4SUN'),
 Timestamp('2012-06-24 00:00:00', freq='WOM-4SUN'),
 Timestamp('2012-07-22 00:00:00', freq='WOM-4SUN'),
 Timestamp('2012-08-26 00:00:00', freq='WOM-4SUN')]

### 11.3.3 移动（超前和滞后）数据

移动（shifting）指的是沿着时间轴将数据前移或后移。

Series和DataFrame都有一个shift方法用于执行数据的前移或后移操作，保持索引不变：


In [43]:
ts = pd.Series(np.random.randn(4),
               index=pd.date_range('1/1/2000', periods=4, freq='M'))

ts

2000-01-31   -0.478810
2000-02-29    0.998316
2000-03-31   -0.308847
2000-04-30    0.162030
Freq: M, dtype: float64

In [46]:
ts.shift(2)
#传入正整数，使得所有数据向后推移

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.478810
2000-04-30    0.998316
Freq: M, dtype: float64

In [47]:
ts.shift(-2)
#传入负整数，使得所有数据向前推移

2000-01-31   -0.308847
2000-02-29    0.162030
2000-03-31         NaN
2000-04-30         NaN
Freq: M, dtype: float64

shift通常用于计算一个时间序列或多个时间序列（如DataFrame的列）中的百分比变化：

In [48]:
ts

2000-01-31   -0.478810
2000-02-29    0.998316
2000-03-31   -0.308847
2000-04-30    0.162030
Freq: M, dtype: float64

In [49]:
ts.shift(1)

2000-01-31         NaN
2000-02-29   -0.478810
2000-03-31    0.998316
2000-04-30   -0.308847
Freq: M, dtype: float64

In [51]:
ts/ts.shift(1) - 1

2000-01-31         NaN
2000-02-29   -3.084996
2000-03-31   -1.309368
2000-04-30   -1.524629
Freq: M, dtype: float64

如果保持数据不变，而是改变索引，需要传入频率参数D。

事实上分别是标签列和数据列的改变方法。

In [52]:
ts

2000-01-31   -0.478810
2000-02-29    0.998316
2000-03-31   -0.308847
2000-04-30    0.162030
Freq: M, dtype: float64

In [53]:
ts.shift(1,freq='D')
#通过传入频率参数freq，来改变索引

2000-02-01   -0.478810
2000-03-01    0.998316
2000-04-01   -0.308847
2000-05-01    0.162030
Freq: MS, dtype: float64

In [54]:
ts.shift(1)
#默认改变数据列

2000-01-31         NaN
2000-02-29   -0.478810
2000-03-31    0.998316
2000-04-30   -0.308847
Freq: M, dtype: float64

## 11.5 时期及其算术运算

时期（period）表示的是时间区间，比如数日、数月、数季、数年等。相较于DateIndex对象不同，period对象是一个更高层次的集合，可以进行向下覆盖的操作。

可以看到，以下生成的时间范围和时期是不同的：

In [60]:
pd.date_range('2000-01', '2000-06',freq='M')

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31'],
              dtype='datetime64[ns]', freq='M')

In [69]:
p = pd.period_range('2000-01', '2000-06',freq='M')
p

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

生成一个Period对象，表示从2007年1月1日到2007年12月31日之间的整段时间：

对Period对象加上或减去一个整数即可达到根据其频率进行位移的效果：

In [62]:
p = pd.Period(2007, freq='A-DEC')
p

Period('2007', 'A-DEC')

In [63]:
p + 5

Period('2012', 'A-DEC')

PeriodIndex对象可以被pandas数据用作轴索引

(注意是pd.period_range方法生成的PeriodIndex对象，而不是简单的pd.Period生成的Period对象)：

In [71]:
ts = pd.Series(np.random.randn(6), index=p)
ts

2000-01   -0.168431
2000-02   -0.581506
2000-03    0.111296
2000-04    0.587888
2000-05    0.714374
2000-06    0.281075
Freq: M, dtype: float64

In [72]:
ts.index
#注意到这里的索引是PeriodIndex，而不是DatetimeIndex。

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

如果有一个字符串数组，也可以使用PeriodIndex类：

In [73]:
values = ['2001Q3', '2002Q2', '2003Q1']
index = pd.PeriodIndex(values, freq='Q-DEC')

index
#PeriodIndex可以当作方法使用，可以将字符串str转换为PeriodIndex对象

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

### 11.5.1 时期的频率转换

Period和PeriodIndex对象都可以通过其asfreq方法被转换成别的频率。

假设我们有一个年度时期，希望将其转换为当年年初或年末的一个月度时期。该任务非常简单：

In [74]:
p = pd.Period('2007', freq='A-DEC')

p

Period('2007', 'A-DEC')

In [75]:
p.asfreq('M', how='start')
#通过asfreq方法，将原Period对象进行了频率转换

Period('2007-01', 'M')

In [76]:
p.asfreq('M', how='end')

Period('2007-12', 'M')

### 11.5.2 Timestamp和Period相互转换

to_period和to_timestamp方法可以实现两个对象之间的转换。

### 11.5.3 通过数组创建PeriodIndex

固定频率的数据集通常会将时间信息分开存放在多个列中。

例如，在下面这个宏观经济数据集中，年度和季度就分别存放在不同的列中。目的是将年份和季度信息整合成时间索引。

In [77]:
data = pd.read_csv(r'E:\Coding\Python\Python exercises\pydata-book-2nd-edition\examples\macrodata.csv')

data.head(5)

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [78]:
data.year

0      1959.0
1      1959.0
2      1959.0
3      1959.0
4      1960.0
        ...  
198    2008.0
199    2008.0
200    2009.0
201    2009.0
202    2009.0
Name: year, Length: 203, dtype: float64

In [80]:
data.quarter

0      1.0
1      2.0
2      3.0
3      4.0
4      1.0
      ... 
198    3.0
199    4.0
200    1.0
201    2.0
202    3.0
Name: quarter, Length: 203, dtype: float64

通过将这些数组以及一个频率传入PeriodIndex，就可以将它们合并成DataFrame的一个索引：

In [81]:
index = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                       freq='Q-DEC')
#注意到PeriodIndex方法有参数year和quarter

index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203, freq='Q-DEC')

In [83]:
data.index = index
#使用新生成的索引

In [85]:
data.head(5)

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
1959Q1,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [86]:
data = data.drop(['year','quarter'],axis=1)
data.head(5)
#将原来的year和quarter两列删除。目标完成。

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
1959Q1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


## 11.6 重采样及频率转换

重采样（resampling）指的是将时间序列从一个频率转换到另一个频率的处理过程。将高频率数据聚合到低频率称为降采样（downsampling），而将低频率数据转换到高频率则称为升采样（upsampling）

pandas对象都带有一个resample方法，它是各种频率转换工作的主力函数。

resample有一个类似于groupby的API，调用resample可以分组数据，然后会调用一个聚合函数。resample的主要参数有：

    freq   采样频率。
    axis   默认为axis=0。
    fill_method    升采样的插值方法。有ffill和bfill，默认不插值。
    closed    降采样中区间的闭端，有left和right，默认是left。
    label    降采样中选取面元的哪段作为标签。例如9：30和9：35这五分钟，会被标记为9：30或9：35。有left和right，默认是left。
    loffset    面元标签矫正值，'-1'或Second(-1)。
    kind    聚合到时期（period）或时间戳(timestamp)。默认聚合到时间序列的索引类型。

In [87]:
rng = pd.date_range('2000-01-01', periods=100, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
#生成范例数据的简单方法是根据索引长度来生成对应长度的随机数，即len(index)。

ts

2000-01-01    0.476630
2000-01-02    0.524871
2000-01-03   -1.667052
2000-01-04    0.190102
2000-01-05   -0.306924
                ...   
2000-04-05    1.230127
2000-04-06   -0.458117
2000-04-07   -0.266888
2000-04-08   -0.687120
2000-04-09   -0.630475
Freq: D, Length: 100, dtype: float64

In [88]:
ts.resample('M').mean()
#通过取平均值的方式进行降采样

2000-01-31    0.127253
2000-02-29   -0.085032
2000-03-31    0.065596
2000-04-30   -0.229355
Freq: M, dtype: float64

In [89]:
ts.resample('M', kind='period').mean()
#这里指定kind参数

2000-01    0.127253
2000-02   -0.085032
2000-03    0.065596
2000-04   -0.229355
Freq: M, dtype: float64

### 11.6.1 降采样

将数据聚合到规律的低频率是一件非常普通的时间序列处理任务。待聚合的数据不必拥有固定的频率，期望的频率会自动定义聚合的面元边界，这些面元用于将时间序列拆分为多个片段。例如，要转换到月度频率（'M'或'BM'），数据需要被划分到多个单月时间段中。各时间段都是半开放的。一个数据点只能属于一个时间段，所有时间段的并集必须能组成整个时间帧。

In [90]:
rng = pd.date_range('2000-01-01', periods=12, freq='T')
#频率T表示分钟

ts = pd.Series(np.arange(12), index=rng)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
                       ..
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, Length: 12, dtype: int32

假设想要通过求和的方式将这些1分钟数据聚合到“5分钟”块中（降采样）。不同的参数设定导致不同的重采样结果。

In [102]:
ts.resample('5T',closed='left').sum()
#面元左闭右开，因此5分钟的增量是以 00/01/02/03/04这样以此类推的5个数据重采样

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5T, dtype: int32

In [103]:
ts.resample('5T',closed='right').sum()
#面元左开右闭，因此5分钟的增量是以 01/02/03/04/05这样以此类推的5个数据重采样。注意到第一个00未被包括到2000年，会算入1999年的最后一个5分钟区间

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5T, dtype: int32

In [105]:
ts.resample('5T',closed='right',label='right').sum()
#上面不指定标签时，默认以left作为标签。可以指定label='right'来使用面元的右边界做标签。

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5T, dtype: int32

最后，可以对结果索引做一些位移，比如从右边界减去一秒以便更容易明白该时间戳到底表示的是哪个区间。

只需通过loffset设置一个字符串或日期偏移量即可实现这个目的：

In [107]:
ts.resample('5T',closed='right',label='right',loffset='1s').sum()
#通过对标签向后位移1s，可以更明白这个5分钟区间属于哪一段。

2000-01-01 00:00:01     0
2000-01-01 00:05:01    15
2000-01-01 00:10:01    40
2000-01-01 00:15:01    11
Freq: 5T, dtype: int32

### 11.6.2 OHLC重采样

金融领域中有一种无所不在的时间序列聚合方式，即计算各面元的四个值：

    第一个值（open，开盘）
    最大值（high，最高）
    最小值（low，最低）
    最后一个值（close，收盘）
    
传入how='ohlc'即可得到一个含有这四种聚合值的DataFrame。整个过程很高效，只需一次扫描即可计算出结果：

In [108]:
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
                       ..
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: T, Length: 12, dtype: int32

In [109]:
ts.resample('5T').ohlc()
#注意，默认以左边界闭合，以左边界作为标签。

,open,high,low,close
2000-01-01 00:00:00,0,4,0,4
2000-01-01 00:05:00,5,9,5,9
2000-01-01 00:10:00,10,11,10,11


### 11.6.3 升采样和插值

从低频率到高频率的升采样，不再需要聚合。而往往需要插值。

In [110]:
frame = pd.DataFrame(np.random.randn(2, 4),
                     index=pd.date_range('1/1/2000', periods=2,
                                         freq='W-WED'),
                     columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,-0.870167,0.841987,0.878198,0.181412
2000-01-12,2.583580,0.632814,0.450799,0.173421


使用asfreq方法升采样：

In [111]:
df_daily = frame.resample('D').asfreq()
#升采样到每天。会产生空值。

df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,-0.870167,0.841987,0.878198,0.181412
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,2.583580,0.632814,0.450799,0.173421


In [115]:
frame.resample('D').ffill(limit=3)
#ffill向前填充。limit限制填充距离。

,Colorado,Texas,New York,Ohio
2000-01-05,-0.870167,0.841987,0.878198,0.181412
2000-01-06,-0.870167,0.841987,0.878198,0.181412
2000-01-07,-0.870167,0.841987,0.878198,0.181412
2000-01-08,-0.870167,0.841987,0.878198,0.181412
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,2.583580,0.632814,0.450799,0.173421


移动窗口函数，后面再补充